# Understanding Consumer Behavior in Supermarkets: A Transactional Analysis
Within this case analysis, SQL has been employed to address queries pertaining to finance and logistics.
As anyone who has ever shopped in a supermarket knows, the checkout experience is an integral part of the retail journey. Every merchant must grasp their point-of-sale statistics to tackle pivotal inquiries such as "Does the payment method influence expenditure?" "What is the average number of items purchased at varying intervals?" and "Are self-service checkouts more expedient than those manned by cashiers?"

## Data Source: 
Retail operations in a grocery supermarket located in a large city in Southern Poland [transaction data from a Polish Supermarket](https://www.mdpi.com/2306-5729/4/2/67/htm)

## Primary Goal: 
Data exploration with SQL for financial analysis

## Secondary Goal: 
Identify trends of sales volume

## Why?
Inform business decisions through financial analysis to increase efficiency and forecast future revenue and workforce requirements. 

## Familiarizing with the database
The data concerns retail operations in a grocery supermarket located in a large city in Southern Poland, equipped with manned (service) and self-service checkouts. The checkout zone is composed of a single waiting line for each service checkout and one waiting line for all self-service checkouts. The data covers three nearly two-week periods: (i) 7 to 19 December 2017; (ii) 13 to 26 February 2019; and (iii) 28 March to 10 April 2019. Please note that the new regulations introduced in Poland in 2018 banned shopping on some Sundays, generally two Sundays per month in 2018 and three Sundays per month in 2019, disrupting the rather regular 7-day pattern observed until the end of 2017. Supermarkets have reacted by extending the working hours on Fridays and Saturdays, while customers had to adapt to the changes in opening hours. The two pairs of datasets from 2019 include one working (24 February, 31 March) and one non-working Sunday (17 February, 7 April) each.

### Select all columns and the first 100 rows from `pos_operator_logs` and `pos_transactions`.


In [4]:
SELECT *
FROM 'pos_operator_logs.csv'
LIMIT 100;

,id,Workstation_Group_ID,Workstation_ID,begin_date_time,operator_id
0,1,8,19,2019-02-13 05:37:55+00:00,269
1,2,8,18,2019-02-13 05:37:55+00:00,268
2,3,8,17,2019-02-13 05:38:43+00:00,267
3,4,1,4,2019-02-13 07:01:26+00:00,332
4,5,1,7,2019-02-13 07:01:57+00:00,10
...,...,...,...,...,...
95,96,1,20,2019-02-13 10:13:59+00:00,328
96,97,1,21,2019-02-13 10:17:43+00:00,338
97,98,1,20,2019-02-13 10:24:06+00:00,328
98,99,1,20,2019-02-13 10:24:24+00:00,328


In [5]:
SELECT *
FROM 'pos_transactions.csv'
LIMIT 100;

,id,WorkstationGroupID,begin_date_time,end_date_time,OperatorID,basket_size,t_cash,t_card,amount
0,1,1,2017-12-07 14:23:23+00:00,2017-12-07 14:24:36+00:00,101,23,True,False,112.71
1,2,1,2017-12-07 14:25:09+00:00,2017-12-07 14:27:00+00:00,101,29,True,False,54.76
2,3,1,2017-12-07 14:27:28+00:00,2017-12-07 14:27:48+00:00,101,3,True,False,14.77
3,4,1,2017-12-07 14:28:04+00:00,2017-12-07 14:28:29+00:00,101,12,True,False,37.88
4,5,1,2017-12-07 14:29:40+00:00,2017-12-07 14:30:32+00:00,101,7,True,False,115.34
...,...,...,...,...,...,...,...,...,...
95,96,1,2017-12-07 17:07:48+00:00,2017-12-07 17:08:39+00:00,101,9,True,False,32.54
96,97,1,2017-12-07 17:10:51+00:00,2017-12-07 17:11:33+00:00,101,9,True,False,26.59
97,98,1,2017-12-07 17:11:47+00:00,2017-12-07 17:12:37+00:00,101,20,True,False,104.82
98,99,1,2017-12-07 17:12:51+00:00,2017-12-07 17:13:58+00:00,101,12,False,True,67.62


## Do more people make transactions by card or by cash?

In [6]:
-- Count of how many transactions were processed by cash (the case when `t_cash` is true) and by card  (the case when `t_card` is true).
SELECT
COUNT (CASE WHEN t_cash THEN 1 END) AS cash_transactions,
COUNT (CASE WHEN t_card THEN 1 END) AS card_transactions
FROM 'pos_transactions.csv'

,cash_transactions,card_transactions
0,84487,78246


## Do people spend more per transaction when using cash or card?

In [7]:
SELECT 
AVG(CASE WHEN t_cash AND NOT t_card THEN amount END) AS avg_cash_transactions,
AVG(CASE WHEN t_card AND NOT t_cash THEN amount END) AS avg_card_transactions
FROM 'pos_transactions.csv';

,avg_cash_transactions,avg_card_transactions
0,58.718519,86.83937


## Exploring Sunday Trading
As mentioned before: the new regulations introduced in Poland in 2018 banned shopping on some Sundays, generally two Sundays per month in 2018 and three Sundays per month in 2019, disrupting the rather regular 7-day pattern observed until the end of 2017. Supermarkets have reacted by extending the working hours on Fridays and Saturdays, while customers had to adapt to the changes in opening hours. The two pairs of datasets from 2019 include one working (24 February, 31 March) and one non-working Sunday (17 February, 7 April) each.

In [8]:
-- Get the week number of the dates where trading occured on a Sunday. 
SELECT
DATE_PART('week', end_date_time:: DATE) AS week_num,
end_date_time::DATE AS end_date
FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) >= '2019'
   AND end_date IN('2019-02-17', '2019-02-24', '2019-03-31', '2019-04-07')
GROUP BY end_date;

,week_num,end_date
0,13,2019-03-31 00:00:00+00:00
1,8,2019-02-24 00:00:00+00:00


Note that the last two dates in my query do not appear and rightly so, because they were non-working Sundays. 
So the Weeks with with working Sundays are 8 and 13 ("WS"). And the weeks with non-working Sundays are 7 and 14 ("NWS"). 

## Analysis of daily trends

In [9]:
-- Total number of transactions from 2019; daily sales amount, daily average amount and median average amount per transaction.
SELECT
DATE_PART('week', end_date_time:: DATE) AS week_num,
end_date_time::DATE AS end_date,
COUNT(id) AS total_transactions,
SUM(amount) AS total_sales,
AVG(amount) AS avg_sale_amount
FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) >= '2019'
GROUP BY end_date ORDER BY week_num;

,week_num,end_date,total_transactions,total_sales,avg_sale_amount
0,7,2019-02-13 00:00:00+00:00,3163,189112.49,59.788963
1,7,2019-02-14 00:00:00+00:00,4404,302242.45,68.629076
2,7,2019-02-15 00:00:00+00:00,4031,319112.44,79.164584
3,7,2019-02-16 00:00:00+00:00,4669,405580.79,86.866736
4,8,2019-02-23 00:00:00+00:00,4367,369412.97,84.591933
5,8,2019-02-18 00:00:00+00:00,3270,193133.29,59.062168
6,8,2019-02-22 00:00:00+00:00,4411,330428.78,74.910175
7,8,2019-02-24 00:00:00+00:00,2258,143025.41,63.341634
8,8,2019-02-20 00:00:00+00:00,3045,181630.10,59.648637
9,8,2019-02-19 00:00:00+00:00,3025,170242.42,56.278486


For further detail I had added the average and median `basket_size` per day in 2019. 

In [10]:
-- Total weekly sales amount for week 8th and 14th in 2019
SELECT
DATE_PART('week', end_date_time:: DATE) AS week_num,
end_date_time::DATE AS end_date,
COUNT(id) AS total_transactions,
SUM(amount) AS total_sales,
AVG(amount) AS avg_sale_amount,
PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY amount) AS median_sale_amount,
AVG(basket_size) AS avg_basket_size,
PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY basket_size) AS median_basket_size
FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM (end_date_time)) >= '2019'
GROUP BY end_date ORDER BY week_num;

,week_num,end_date,total_transactions,total_sales,avg_sale_amount,median_sale_amount,avg_basket_size,median_basket_size
0,7,2019-02-13 00:00:00+00:00,3163,189112.49,59.788963,37.580,12.915903,9.0
1,7,2019-02-14 00:00:00+00:00,4404,302242.45,68.629076,46.285,14.172116,10.0
2,7,2019-02-15 00:00:00+00:00,4031,319112.44,79.164584,50.900,17.184818,12.0
3,7,2019-02-16 00:00:00+00:00,4669,405580.79,86.866736,57.600,19.733348,14.0
4,8,2019-02-23 00:00:00+00:00,4367,369412.97,84.591933,56.370,19.813144,14.0
5,8,2019-02-18 00:00:00+00:00,3270,193133.29,59.062168,37.420,13.810092,10.0
6,8,2019-02-22 00:00:00+00:00,4411,330428.78,74.910175,49.060,17.736568,12.0
7,8,2019-02-24 00:00:00+00:00,2258,143025.41,63.341634,37.660,15.169176,10.0
8,8,2019-02-20 00:00:00+00:00,3045,181630.10,59.648637,36.560,13.594417,9.0
9,8,2019-02-19 00:00:00+00:00,3025,170242.42,56.278486,37.530,13.432727,9.0


## Should the supermarket open on Sundays?
The two Sundays I see are significantly lower in volume of transactions than all other days. 
Also, Thursdays, Fridays and Saturdays are approximately 25% higher in number of transactions and 40% higher in sales revenue than Mondays, Tuesdays and Wednesdays.

Does it make sense to open stores on Sundays? 
As we only have full week data available for week 8 (WS) and week 14 (NWS). I will compare these two to answer this concern.

In [11]:
SELECT 
DATE_PART('week', end_date_time::DATE) as week_num,
SUM(amount)
FROM 'pos_transactions.csv'

WHERE EXTRACT(YEAR FROM end_date_time) >= '2019' AND week_num IN ('8','14')

GROUP BY week_num

,week_num,sum(amount)
0,8,1686485.72
1,14,1690989.87


## How much does labor cost?
Week 8 (WS) revenue is PLN1.7M and also week 14 (NWS) has PLN1.7M. It may be possible that customers go shopping before Sunday knowing that the store will be closed on that day. 

Assuming that the competition is going to also notice this trend and close on Sundays, and assuming that our stores can rearrange the workforce shifts to 6 days a week cutting the Sunday shifts **there may be possible savings from closing the stores on Sundays with no impact on revenue**. 

To go deeper on the analysis I will start quantifying the savings with labour costs. The question I want to answer is: how many operators were working on working Sundays? 

In [12]:
-- Num of operators working on each day in 2019 by counting operator_id working on each being_date_time
SELECT 
(begin_date_time::DATE) as working_day,
COUNT(DISTINCT(operator_id)),
DATE_PART('week',(begin_date_time::DATE)) as week_num

FROM 'pos_operator_logs.csv'

WHERE EXTRACT(YEAR FROM begin_date_time) >= '2019'

GROUP BY working_day

,working_day,count(DISTINCT operator_id),week_num
0,2019-02-13 00:00:00+00:00,20,7
1,2019-02-14 00:00:00+00:00,26,7
2,2019-02-15 00:00:00+00:00,24,7
3,2019-02-16 00:00:00+00:00,28,7
4,2019-02-19 00:00:00+00:00,21,8
5,2019-02-21 00:00:00+00:00,27,8
6,2019-02-25 00:00:00+00:00,20,9
7,2019-02-26 00:00:00+00:00,20,9
8,2019-03-28 00:00:00+00:00,25,13
9,2019-03-30 00:00:00+00:00,26,13


After analyzing the data, it was found that there were 18 employees working on Sunday, February 24th, and 20 employees working on Sunday, March 31st. Moving forward, further investigation will focus on determining the hours per shift and the cost per hour to calculate the average savings per Sunday. Additionally, all other costs associated with keeping stores open and operational, such as energy expenses, will be estimated. It's important to note that while closing stores for a day may result in potential savings, there may also be additional costs incurred, such as losses from unsold short-shelf-life goods. These factors will be carefully considered in future decision-making processes.